# Ερώτημα 2 (Α2)

In [1]:
import pandas as pd
from ast import literal_eval
import gmplot
from fastdtw import fastdtw
from math import radians, cos, sin, asin, sqrt
from scipy.spatial.distance import euclidean
import numpy as np
import os
import time

def haversine(A, B):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1 = A[0]
    lat1 = A[1]
    lon2 = B[0]
    lat2 = B[1]
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def LCS(X, Y):
    threshold = 0.2
    m = len(X)
    n = len(Y)
    # An (m+1) times (n+1) matrix
    C = [[0] * (n + 1) for _ in range(m + 1)]
    #print C
    for i in range(1, m+1):
        for j in range(1, n+1):
            if haversine(X[i-1],Y[j-1]) <= threshold:
                C[i][j] = C[i-1][j-1] + 1
            else:
                C[i][j] = max(C[i][j-1], C[i-1][j])
    return C

def backTrack(C, X, Y, i, j):
    threshold = 0.2
    if i == 0 or j == 0:
        return []
    if haversine(X[i-1],Y[j-1]) <= threshold:
        l =backTrack(C, X, Y, i-1, j-1)
        l.append(X[i-1].tolist())
        return  l
    else:
        if C[i][j-1] > C[i-1][j]:
            return backTrack(C, X, Y, i, j-1)
        else:
            return backTrack(C, X, Y, i-1, j)

In [3]:
train_set = pd.read_csv('datasets/train_set.csv',
                        converters={"Trajectory": literal_eval},
                        index_col='tripId')

test_set1 = pd.read_csv('datasets/test_set_a2.csv',
                        sep='\t',
                        converters={"Trajectory": literal_eval})

In [4]:
Trainnp = []
for x  in train_set["Trajectory"]:
    temp1 = np.asarray(x)
    temp1 = temp1[:, [1,2]]
    Trainnp.append(temp1)
    
Test1np = []
for x  in test_set1["Trajectory"]:
    temp = np.asarray(x)
    temp = temp[:, [1,2]]
    Test1np.append(temp)
    
Ids = []
for x in train_set['journeyPatternId']:
    Ids.append(x)  

In [5]:
if not os.path.exists("./erwthma2_A2"):
    os.makedirs("./erwthma2_A2")
k=1  
for x in Test1np:
    start = time.time()
    time.clock() 
    pathlens = [0] * 5
    ids =[0] * 5
    paths = [0] * 5
    minpos = 0
    minval = -1
    temp_id=0
    for y in Trainnp:
        C = LCS(x,y)
        i = len(x)
        j = len(y)
        path = backTrack(C,x,y,i,j)
        if len(path) > minval:
            pathlens[minpos] = len(path)
            paths[minpos] = path
            ids[minpos] = Ids[temp_id]
            minval = min(pathlens)
            minpos = pathlens.index(minval)
        temp_id +=1
    print "Longest common path lengths of the five neighbours of Test trip " +str(k)+" :"
    print "\t",pathlens
    print "\n"
    print "Journey Pattern Ids of the five neighbours of Test trip " +str(k)+" :"
    print "\t",ids
    print "\n"
    
    if not os.path.exists("./erwthma2_A2/Test_trip_"+str(k)):
        os.makedirs("./erwthma2_A2/Test_trip_"+str(k))
        
    longitudes=[]
    latitudes=[]
    
    for point in x:
        longitudes.append(point[0])
        latitudes.append(point[1])
    gmap = gmplot.GoogleMapPlotter(latitudes[0],longitudes[0],16)
    gmap.plot(latitudes, longitudes, 'green', edge_width=10)
    map_name=("./erwthma2_A2/Test_trip_"+str(k)+"/Test_trip_"+str(k)+".html")
    gmap.draw(map_name)
    j=1
    for neighbour in paths:
        '''
        longitudes=[]
        latitudes=[]
        for point in x:
            longitudes.append(point[0])
            latitudes.append(point[1])
        '''
        gmap = gmplot.GoogleMapPlotter(latitudes[0],longitudes[0],16)
        gmap.plot(latitudes, longitudes, 'green', edge_width=10)
        longitudes1=[]
        latitudes1=[]
        for point in neighbour:
            longitudes1.append(point[0])
            latitudes1.append(point[1])
        gmap.plot(latitudes1, longitudes1, 'red', edge_width=7)
        map_name=("./erwthma2_A2/Test_trip_"+str(k)+"/Neighbour_"+str(j)+".html")
        gmap.draw(map_name)
        j+=1
    elapsed = time.time() - start
    print "elapsed time for Test Trip "+str(k)+": ",elapsed,"\n"
    k+=1
    

Longest common path lengths of the five neighbours of Test trip 1 :
	[78, 76, 75, 78, 76]


Journey Pattern Ids of the five neighbours of Test trip 1 :
	['040D1002', '040D1002', '040D1002', '040D1002', '040D1002']


elapsed time for Test Trip 1:  388.475136995 

Longest common path lengths of the five neighbours of Test trip 2 :
	[73, 78, 74, 75, 82]


Journey Pattern Ids of the five neighbours of Test trip 2 :
	['040D1002', '040D1002', '040D1002', '040D1002', '040D1002']


elapsed time for Test Trip 2:  403.157147169 

Longest common path lengths of the five neighbours of Test trip 3 :
	[40, 40, 40, 40, 40]


Journey Pattern Ids of the five neighbours of Test trip 3 :
	['01451001', '00790001', '067X0001', '079A0001', '01451008']


elapsed time for Test Trip 3:  200.924605131 

Longest common path lengths of the five neighbours of Test trip 4 :
	[59, 59, 59, 59, 59]


Journey Pattern Ids of the five neighbours of Test trip 4 :
	['00790001', '00790001', '00790001', '00790001', '00790001